# 1 : Librairies Python

In [1]:
# pip install pandas sqlalchemy psycopg2-binary schedule

In [2]:
# Librairies et options :

import pandas as pd
import random
import datetime
import requests
import json
import numpy as np
import tqdm
import time

from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.types import Integer, BigInteger, JSON

# 2 : Ouverture de l'API

In [3]:
# Informations : https://weatherlink.github.io/v2-api/

APIKey = 'zjbvtfdvjslxklxclzscfu6isqpqfxbg'
APISecret = 'mih0udjsuetwaatpwzrqzeglljmy889b'
StationID = 122495

In [4]:
# Date d'aujourd'hui à minuit :
Today = datetime.date.today()
TodayMidnight = datetime.datetime.combine(Today, datetime.time.min)

# Date du début de la station :
StartStation = datetime.datetime(2021, 10, 1, 0, 0)

# Convertir la date en timestamp
StartStationTimestamp = int(StartStation.timestamp())
TodayMidnightTimestamp = int(TodayMidnight.timestamp())


print("Le timestamp correspondant à la date du {} est :".format(TodayMidnight), TodayMidnightTimestamp)
print("Le timestamp correspondant à la date du {} est :".format(StartStation), StartStationTimestamp)

Le timestamp correspondant à la date du 2024-04-13 00:00:00 est : 1712959200
Le timestamp correspondant à la date du 2021-10-01 00:00:00 est : 1633039200


In [5]:
# DataFrame historiques :
dfHistorique = pd.DataFrame()

# Nb de jours à récupérer :
NbJours = int((TodayMidnightTimestamp - StartStationTimestamp) / 86400)

for i in tqdm.tqdm(range(NbJours)):
    StartTime = StartStationTimestamp + i * 86400
    EndTime = StartTime + 86400
    
    # Lien de la request : 
    link = 'https://api.weatherlink.com/v2/historic/{}?api-key={}&start-timestamp={}&end-timestamp={}'.format(StationID, APIKey, StartTime, EndTime)
    headers = {'X-Api-Secret' : APISecret}

    # Requête :
    r = requests.get(link, headers=headers)

    # Si la requête a réussi :
    if r.status_code == 200:
        # Lecture de la request en json :
        data = r.json()

        # Transformation en DF : 
        dfJour = pd.DataFrame(data)
        dfJour = dfJour[['station_id','sensors']]

        # Récupération des valeurs se trouvant dans sensors :
        dfSensors = pd.json_normalize(data['sensors'][0]['data'])
    
        # Récupération des json sur une colonne :
        dfJour = pd.DataFrame({'station_id': data['station_id'], 'infos_json' : data['sensors'][0]['data']})
    
        # Convertir les objets JSON en chaînes de caractères JSON :
        dfJour['infos_json'] = dfJour['infos_json'].apply(json.dumps)

        # Concat des données :
        dfJour = pd.concat([dfJour, dfSensors], axis=1)
        
        # Concaténation des données :
        dfHistorique = pd.concat([dfHistorique, dfJour], ignore_index=True)
    else:
        print("La requête {} a échoué avec le code d'erreur {}".format(link, r.status_code))

 36%|████████████████████████████▉                                                   | 334/925 [22:42<40:10,  4.08s/it]


KeyboardInterrupt: 

In [ ]:
# Data Frame Finale :
dfHistorique.head()

# 3 : Transfert sur PostgreSQL

In [ ]:
# Paramètres de connexion à la base de données PostgreSQL en local :
host = "localhost"
database = "Test_LPO"
user = "postgres"
password = "15072022"

# Début du chronomètre :
start_time = time.time()

# Création de la chaîne de connexion PostgreSQL :
conn_str = f"postgresql://{user}:{password}@{host}/{database}"

# Création de la connexion à la base de données PostgreSQL :
engine = create_engine(conn_str)

# Définir les types de données pour chaque colonne :
dtype = {'station_id': Integer(),
         'ts': BigInteger(),
         'infos_json': JSON}

# Insérer le DataFrame dans la base de données PostgreSQL :
dfHistorique.to_sql('historiquemeteo', engine, if_exists = 'replace', index=False, dtype=dtype)

# Fermeture de la connexion :
engine.dispose()

# Fin du chronomètre :
end_time = time.time()

# Calcul de la durée d'exécution :
execution_time = end_time - start_time

# Affichage de la durée d'exécution :
print("Durée d'exécution :", round(execution_time/60,2), "minutes")

# 4 : Ouverture de la BDD

In [ ]:
# Connexion à la base de données
conn = psycopg2.connect(dbname = database, user = user, password = password , host = host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour sélectionner les données de ma_table
cur.execute("SELECT * FROM historiquemeteo LIMIT 5")

# Récupération des données dans une liste de tuples
data = cur.fetchall()

# Création d'un DataFrame à partir des données
df = pd.DataFrame(data, columns=[desc[0] for desc in cur.description])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

df

In [ ]:
import psycopg2

# Connexion à la base de données
conn = psycopg2.connect(dbname=database, user=user, password=password, host=host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour obtenir les informations de schéma de la table
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'historiquemeteo'")

# Récupération des données dans une liste de tuples
column_info = cur.fetchall()

# Affichage du format de chaque colonne
for column in column_info:
    print("La colonne ", column[0], "à pour format :", column[1])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()